In [0]:
tiers = ["bronze", "silver", "gold"]

adls_paths = {tier: f"abfss://{tier}@earthquakedatalake1.dfs.core.windows.net/" for tier in tiers}

bronze_adls=adls_paths["bronze"]
silver_adls=adls_paths["silver"]
gold_adls=adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
start_date, end_date

(datetime.date(2025, 4, 16), datetime.date(2025, 4, 17))

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

In [0]:
response = requests.get(url)
response.raise_for_status()
data=response.json().get('features',[])

In [0]:
if not data:
    print("No data returned for the specified date range.")
file_path=f"{bronze_adls}/{start_date}_earthquake_data.json"
# Convert each record to a JSON string and write all to a newline-delimited string
json_lines = "\n".join([json.dumps(record) for record in data])

# Save it in a way that Spark can read
dbutils.fs.put(file_path, json_lines, overwrite=True)


Wrote 232033 bytes.


True

In [0]:
try:    
    print(f"Data successfully saved to {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")


Data successfully saved to abfss://bronze@earthquakedatalake1.dfs.core.windows.net//2025-04-16_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 2.22,
  'place': '22 km NNE of California City, CA',
  'time': 1744847722170,
  'updated': 1744847929679,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci40930655',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci40930655&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 76,
  'net': 'ci',
  'code': '40930655',
  'ids': ',ci40930655,',
  'sources': ',ci,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 44,
  'dmin': 0.07462,
  'rms': 0.18,
  'gap': 58,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 2.2 - 22 km NNE of California City, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-117.9233333, 35.3216667, 7.05]},
 'id': 'ci40930655'}